In [16]:
from PIL import Image, ImageDraw, ImageFont
import glob
import os

def annotate_braf_msi(
    input_path: str,
    lines=(),
    output_path: str = None,
    font_path: str = None,
    font_size: int = 100,
    margin: int = 10,
    padding: int = 5,
    box_fill=(255, 255, 255, 200),
    text_fill=(0, 0, 0, 255),
):
    """
    Draw a semi‑opaque box + text in the top‑right corner of an image,
    using textbbox() to measure each line.

    – input_path:  path to your PNG
    – output_path: where to save (defaults to overwrite)
    – lines:       iterable of strings, one per line
    – font_path:   .ttf file (falls back to default)
    – font_size:   in points
    – margin:      px from top/right edge
    – padding:     px between text and box edge
    – box_fill:    RGBA rectangle fill
    – text_fill:   RGBA text color
    """
    im = Image.open(input_path).convert("RGBA")
    draw = ImageDraw.Draw(im, "RGBA")

    # Load font
    if font_path and os.path.exists(font_path):
        font = ImageFont.truetype(font_path, font_size)
    else:
        font = ImageFont.load_default()

    # Measure each line’s width & height via textbbox
    widths = []
    heights = []
    for line in lines:
        bbox = draw.textbbox((0, 0), line, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        widths.append(w)
        heights.append(h)

    text_block_w = max(widths)
    text_block_h = sum(heights) + padding * (len(lines) - 1)

    # Compute rectangle coords anchored to top‑right
    x1 = im.width  - margin
    y1 = margin
    x0 = x1 - (text_block_w  + 2 * padding)
    y0 = y1 + (text_block_h + 2 * padding)

    # Draw semi‑transparent box
    draw.rectangle([(x0, y1), (x1, y0)], fill=box_fill)

    # Draw each line
    y = y1 + padding
    for line, h in zip(lines, heights):
        draw.text((x0 + padding, y), line, font=font, fill=text_fill)
        y += h + padding

    # Save (overwrite by default)
    out = output_path or input_path
    im.save(out)


In [21]:
# ————————————
# Batch‑process all your renamed PNGs:
cond = "braf0_msi1"
mut = "MSI"
image_folder = f"CLAM/heatmap_outputs/{mut}/{cond}"
for path in glob.glob(os.path.join(image_folder, f"*{cond}.png")):
    annotate_braf_msi(path, lines=(f"{mut} Heatmap", "BRAF: negative", "MSI: positive"),)